In [ ]:
import torch
import torch.onnx
import onnx
from torch.nn.modules.utils import _pair
from torch.onnx import register_custom_op_symbolic
from EvEye.model.DavisEyeEllipse.EPNet.EPNet import EPNet
from EvEye.model.DavisEyeCenter.TennSt import TennSt
from EvEye.model.DavisEyeEllipse.ElNet.ElNet import Creat_MyNet
# from EvEye.model.DavisEyeEllipse.UNet.UNet import UNet

In [ ]:
# model_path = "/mnt/data2T/junyuan/eye-tracking/logs/EPNet_FixedCount5000_TrigERAugFPNdw/version_0/checkpoints/epoch=53-val_mean_distance=0.2249.ckpt"
# output_path = "/mnt/data2T/junyuan/eye-tracking/outputs/EPNetFPNdw.onnx"
# ckpt = torch.load(model_path)
# model = EPNet(input_channels=2,head_dict= { "hm": 1, "ab": 2, "trig": 2, "reg": 2, "mask": 1 },mode="fpn_dw")
# model.load_state_dict(ckpt['state_dict'])
# model.eval()
# dummy_input = torch.randn(1, 2, 256, 256)

In [ ]:
model_path = "/mnt/data2T/junyuan/eye-tracking/weights/96.89%-FixedCount5000-Down-Aug-NoFlip/checkpoints/epochepoch=47-val_p10_accval_p10_acc=0.9689.ckpt"
output_path = "/mnt/data2T/junyuan/eye-tracking/outputs/TennSt.onnx"
ckpt = torch.load(model_path)
model = TennSt(
    channels=[2, 8, 16, 32, 48, 64, 80, 96, 112, 128, 256],
    t_kernel_size=5,
    n_depthwise_layers=4,
    detector_head=True,
    detector_depthwise=True,
    full_conv3d=False,
    norms="mixed",
    activity_regularization=0,
)
model.load_state_dict(ckpt['state_dict'])
model.eval()
dummy_input = torch.rand(1, 2, 1, 256, 256)

In [ ]:
# def dcnv2_symbolic(g, input, offset, mask, weight, bias, stride, padding, dilation, deformable_groups):
#     stride = _pair(stride)
#     padding = _pair(padding)
#     dilation = _pair(dilation)
#     return g.op(
#         "DCNv2_2",
#         input,
#         offset,
#         mask,
#         weight,
#         bias,
#         stride_i=stride,
#         padding_i=padding,
#         dilation_i=dilation,
#         deformable_groups_i=deformable_groups,
#     )

# register_custom_op_symbolic('::DCNv2_2', dcnv2_symbolic, 17)

# model_path = "/mnt/data2T/junyuan/eye-tracking/logs/ElNet_FixedCount5000/version_0/checkpoints/epoch=00-val_mean_distance=0.6273.ckpt"
# output_path = "/mnt/data2T/junyuan/eye-tracking/outputs/ElNet.onnx"
# ckpt = torch.load(model_path)
# model = Creat_MyNet(
#         base_name="dla34",
#         heads={"hm": 1, "ab": 2, "ang": 1, "trig": 2, "reg": 2, "mask": 1},
#         pretrained=True,
#         down_ratio=4,
#         final_kernel=1,
#         last_level=5,
#         head_conv=256,
#         out_channel=0,
#     )
# model.load_state_dict(ckpt['state_dict'])
# model.eval()
# dummy_input = torch.randn(1, 3, 256, 256)

In [ ]:
# model_path = "/mnt/data2T/junyuan/eye-tracking/logs/RGBUNet/version_0/checkpoints/epoch=27-val_mean_distance=0.3231.ckpt"
# output_path = "/mnt/data2T/junyuan/eye-tracking/outputs/UNet.onnx"
# ckpt = torch.load(model_path)
# model = UNet(n_channels=1,n_classes=2)
# model.load_state_dict(ckpt['state_dict'])
# model.eval()
# dummy_input = torch.randn(1, 1, 256, 256)

In [ ]:
torch.onnx.export(
    model,               # 模型
    dummy_input,         # 示例输入
    output_path,  # 输出文件名
    export_params=True,  # 存储模型参数
    opset_version=17,    # ONNX 版本
    do_constant_folding=True,  # 是否执行常量折叠优化
    input_names=['input'],    # 输入名称
    output_names=['output'],   # 输出名称
)

In [1]:
import torch
from torch import nn
from torch.nn.modules.utils import _pair
import _ext as _backend
from DCNv2.dcn_v2 import _DCNv2
from EvEye.model.DavisEyeEllipse.ElNet.ElNet import Creat_MyNet
# 注册自定义操作符
# from torch.onnx import register_custom_op_symbolic
# register_custom_op_symbolic('mydomain::DCNv2_2', _DCNv2.symbolic, 17)

# 加载模型和权重
model_path = "/mnt/data2T/junyuan/eye-tracking/logs/ElNet_FixedCount5000/version_0/checkpoints/epoch=00-val_mean_distance=0.6273.ckpt"
ckpt = torch.load(model_path)
model = Creat_MyNet(
    base_name="dla34",
    heads={"hm": 1, "ab": 2, "ang": 1, "trig": 2, "reg": 2, "mask": 1},
    pretrained=True,
    down_ratio=4,
    final_kernel=1,
    last_level=5,
    head_conv=256,
    out_channel=0,
)
model.load_state_dict(ckpt['state_dict'])
model.eval()

# 示例输入
dummy_input = torch.randn(1, 3, 256, 256)

# 导出模型
output_path = "/mnt/data2T/junyuan/eye-tracking/outputs/ElNet.onnx"
torch.onnx.export(
    model,
    dummy_input,
    output_path,
    export_params=True,
    opset_version=17,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    # verbose=True,
    operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK,
)

/home/junyuan/anaconda3/envs/EvEye/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/data2T/junyuan/eye-tracking/EvEye/model/DavisEyeEllipse/ElNet/ElNet.py:826: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if train:


In [3]:
import onnx

onnx_model = onnx.load("/mnt/data2T/junyuan/eye-tracking/outputs/ElNet.onnx")
onnx.checker.check_model(onnx_model)
print("No Error")
# print(onnx.helper.printable_graph(onnx_model.graph))

ValidationError: No Op registered for DCNv2 with domain_version of 17

==> Context: Bad node spec for node. Name: /dla_up/ida_0/proj_1/conv/DCNv2 OpType: DCNv2